In [ ]:
# This notebook is used to parse the results of the plans generated by planGPT

from metric import *
import pandas as pd
import os
def parse_problems(domains_list):
    result_domain_addition = {}
    df = pd.DataFrame(columns=['domain', 'type', 'total_plans', 'true_plans', 'false_plans', 'accuracy'])

    for test in domains_list:
        if test[0] == "satellite" and test[1] == "new_eval_with_types/test_beam_1/0_actions/":
            dict_actions_domain['satellite'] = dict_actions_s_invariant
        elif test[0] == "satellite":
            dict_actions_domain['satellite'] = dict_actions_s_normal
        domain = test[0]
        addition = test[1]
        result_list = []

        folder_data_path = "../data_for_paper/generations/" + domain + "/" + addition 
        # NOTE: it's necessary to download this folder from Zenodo to see this results: 
        # you can also generate them by yourself using the plans_generator.py script

        if not os.path.exists(folder_data_path):
            print(f"Folder not found {folder_data_path}\n")
            continue
        for file in glob.iglob(f'{folder_data_path}*'):
            with open(file) as json_file:
                if file.endswith(".json"):
                    data = json.load(json_file)
                    for plan in data:
                        if type(plan['plan']) == list: # This check turns true if we have multiple plans generated by planGPT
                            tmp_results = []
                            for i in range(len(plan['plan'])):
                                tmp_plan = {}
                                tmp_plan['actions'] = plan['plan'][i]
                                tmp_plan['input'] = plan['input']
                                tmp_res = parse_problem(tmp_plan, domain)
                                tmp_results.append((tmp_plan, tmp_res))
                        
                            min_plan = None
                            for x in tmp_results:
                                p, res = x
                                if res[0] is True:
                                    if min_plan is None or len(p['actions']) < len(min_plan[0]['actions']):
                                        min_plan = x
                            if min_plan is not None:
                                result_list.append(min_plan)
                            else:
                                result_list.append(tmp_results[0])
                        else:
                        
                            plan['actions'] = remove_blanks(plan['plan'])
                            plan['input'] = remove_blanks(plan['input'])           
                            result = parse_problem(plan, domain)
                            result_list.append((plan, result))
            
        true_plans = []
        false_plans = []
        for result in result_list:
            if result[1][0] is True:
                true_plans.append(result)
            elif result[1][0] is False:
                false_plans.append(result)

        print("Test on " + domain + " " + addition + ' ' + test[2])
        print("Total number of test's plans: " + str(len(result_list)))
        print("Number of plans that satisfy all goals: " + str(len(true_plans)))
        print("Number of invalid plans: " + str(len(false_plans)))
        try:
            df.loc[len(df)] = [domain, test[2], len(result_list), len(true_plans), len(false_plans), len(true_plans)*100/len(result_list)]
            print("Accuracy: " + str(len(true_plans)*100/len(result_list)))
        except:
            df.loc[len(df)] = [domain, test[2], 0, 0, 0, 0]
            print("Accuracy: 0.0")
        print("-"*50)
        result_domain_addition[domain+" "+addition] = (result_list, true_plans, false_plans)
    return result_domain_addition, df
domains_name = ["blocksworld", "depots", "driverlog", "floortile", "logistics", "satellite", "visitall", "zenotravel"]
domains_list = []
for domain in domains_name:
    # For each domain we calculate results for the folder ../data_for_paper/generations/DOMAIN/max_metric/greedy/0_actions/ ...etc.
    domains_list.append((domain, "max_metric/greedy/0_actions/", "max metric greedy generation"))
    domains_list.append((domain, "max_metric/sampling_10_seqs/0_actions_top_p_0.9/", "max metric top p generation"))
    domains_list.append((domain, "max_metric/multibeam_best/0_actions/", "max metric multibest generation"))

    domains_list.append((domain, "min_loss/greedy/0_actions/", "min loss greedy generation"))
    domains_list.append((domain, "min_loss/sampling_10_seqs/0_actions_top_p_0.9/", "min loss top p generation"))
    domains_list.append((domain, "min_loss/multibeam_best/0_actions/", "min loss multibest generation"))

result_domain_addition, df = parse_problems(domains_list)

In [ ]:
from IPython.display import display
# In this cell we display the results of the experiments
df_no_zero = df[df['total_plans'] != 0]
'''for domain in df['domain'].unique():
    tmp = df[df['domain'] == domain]
    display(tmp)'''
display(df_no_zero)
display(df)